## Importing all the libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
keras = tf.keras
import plotly.graph_objects as go

## Helper Functions

In [3]:
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

## Importing CoronaVirus Cases Data

In [45]:
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
death_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
recovered_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')

## Data Pre Processing

### Confirmed Data

In [5]:
confirmed_df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/3/20,8/4/20,8/5/20,8/6/20,8/7/20,8/8/20,8/9/20,8/10/20,8/11/20,8/12/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,36747,36782,36829,36896,37015,37054,37054,37162,37269,37345
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,5620,5750,5889,6016,6151,6275,6411,6536,6676,6817
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,31972,32504,33055,33626,34155,34693,35160,35712,36204,36699
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,937,939,939,944,955,955,955,963,963,977
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,1280,1344,1395,1483,1538,1572,1672,1679,1735,1762


In [6]:
death_df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/3/20,8/4/20,8/5/20,8/6/20,8/7/20,8/8/20,8/9/20,8/10/20,8/11/20,8/12/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,1288,1288,1294,1298,1307,1312,1312,1328,1344,1354
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,172,176,182,188,189,193,199,200,205,208
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,1239,1248,1261,1273,1282,1293,1302,1312,1322,1333
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,52,52,52,52,52,52,52,52,52,53
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,58,59,62,64,67,70,75,78,80,80


In [7]:
recovered_df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/3/20,8/4/20,8/5/20,8/6/20,8/7/20,8/8/20,8/9/20,8/10/20,8/11/20,8/12/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,25669,25669,25742,25840,25903,25960,25960,26228,26415,26694
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,3031,3031,3123,3155,3227,3268,3342,3379,3480,3552
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,21901,22375,22802,23238,23667,24083,24506,24920,25263,25627
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,821,825,825,828,839,839,839,839,839,855
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,476,503,506,520,544,564,567,569,575,577


In [46]:
confirmed_df = confirmed_df.iloc[:, 4:]
confirmed_df = confirmed_df.sum(axis=0)
confirmed_df.index = pd.to_datetime(confirmed_df.index).strftime('%Y-%m-%d')
confirmed_df = pd.DataFrame(confirmed_df)
confirmed_df.head()

,0
2020-01-22,555
2020-01-23,654
2020-01-24,941
2020-01-25,1434
2020-01-26,2118


### Death Data

In [9]:
death_df = death_df.iloc[:, 4:]
death_df = death_df.sum(axis=0)
death_df.index = pd.to_datetime(death_df.index).strftime('%Y-%m-%d')
death_df = pd.DataFrame(death_df)

### Recovered Data

In [10]:
recovered_df = recovered_df.iloc[:, 4:]
recovered_df = recovered_df.sum(axis=0)
recovered_df.index = pd.to_datetime(recovered_df.index).strftime('%Y-%m-%d')
recovered_df = pd.DataFrame(recovered_df)

# Overall Corona Cases

## Preparing Data For Time Series Analysis Using LSTM

In [57]:
time_steps = 1
X_train_confirm, y_train_confirm = create_dataset(confirmed_df, confirmed_df, time_steps)
X_train_death, y_train_death = create_dataset(death_df, death_df, time_steps)
X_train_recovered, y_train_recovered = create_dataset(recovered_df, recovered_df, time_steps)

## Model Building

## Model for Confirmed Cases

In [58]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

confirm_model = keras.Sequential()
confirm_model.add(keras.layers.LSTM(units=128,input_shape=(X_train_confirm.shape[1], X_train_confirm.shape[2]),return_sequences = True))
confirm_model.add(keras.layers.Dropout(rate=0.2))
confirm_model.add(keras.layers.Dense(units=1))
confirm_model.compile(loss='mean_squared_error', optimizer='adam')
reset_states = ResetStatesCallback()
history = confirm_model.fit(X_train_confirm, y_train_confirm,epochs=500,batch_size=16,shuffle=False,callbacks = [reset_states],verbose = 0)


In [18]:
confirm_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 1, 128)            66560     
_________________________________________________________________
dropout (Dropout)            (None, 1, 128)            0         
_________________________________________________________________
dense (Dense)                (None, 1, 1)              129       
Total params: 66,689
Trainable params: 66,689
Non-trainable params: 0
_________________________________________________________________


## Model for Death Cases

In [19]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

death_model = keras.Sequential()
death_model.add(keras.layers.LSTM(units=128,input_shape=(X_train_death.shape[1], X_train_death.shape[2]),return_sequences = True))
death_model.add(keras.layers.Dropout(rate=0.2))
death_model.add(keras.layers.Dense(units=1))
death_model.compile(loss='mean_squared_error', optimizer='adam') 
reset_states = ResetStatesCallback()
death_model.fit(X_train_death, y_train_death,epochs=500,batch_size=16,shuffle=False,callbacks = [reset_states],verbose=0)


## Model for Recovered Cases

In [20]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

recovered_model = keras.Sequential()
recovered_model.add(keras.layers.LSTM(units=128,input_shape=(X_train_recovered.shape[1], X_train_recovered.shape[2]),return_sequences = True))
recovered_model.add(keras.layers.Dropout(rate=0.2))
recovered_model.add(keras.layers.Dense(units=1))
recovered_model.compile(loss='mean_squared_error', optimizer='adam') 
reset_states = ResetStatesCallback()
recovered_model.fit(X_train_recovered, y_train_recovered,epochs=500,batch_size=16,shuffle=False,callbacks = [reset_states],verbose=0)


## Predicting The Results

In [22]:
DAYS_TO_PREDICT = int(input('Enter Number Of Days You Want to Predict : '))

Enter Number Of Days You Want to Predict : 13


### Predicting Results For Confirmed Cases

In [24]:
confirm_test_seq  = y_train_confirm[-1:]
confirm_preds = []
for _ in range(DAYS_TO_PREDICT):
    y2_confirm = confirm_test_seq.reshape((len(confirm_test_seq ), time_steps, 1))
    confirm_pred = confirm_model.predict(y2_confirm)
    confirm_preds.append(confirm_pred)
    confirm_new_seq = confirm_test_seq.flatten()
    confirm_new_seq = np.append(confirm_new_seq,[confirm_pred])
    confirm_new_seq = confirm_new_seq[-1:]
    confirm_test_seq = confirm_new_seq.reshape((len(confirm_new_seq), time_steps, 1))
    y2_confirm = confirm_test_seq
    
confirm_preds = (np.array(confirm_preds).flatten()).reshape(-1,1)

predicted_confirmed_index = pd.date_range(start=confirmed_df.index[-1],periods=DAYS_TO_PREDICT + 1,closed='right').strftime('%Y-%m-%d')
predicted_confirmed_cases = pd.DataFrame()
predicted_confirmed_cases['Dates'] = predicted_confirmed_index
predicted_confirmed_cases['Confirmed Cases Predictions'] = confirm_preds
predicted_confirmed_cases.set_index('Dates', inplace = True)

In [26]:
fig = go.Figure()
#fig.update_layout(template='plotly_dark')
fig.update_xaxes(tickangle=90, showticklabels = True, type = 'category')
fig.add_trace(go.Scatter(x=predicted_confirmed_cases.index, 
                         y=predicted_confirmed_cases['Confirmed Cases Predictions'],
                         mode='lines+markers',
                         name='confirm',
                         line=dict(color='red', width=2)))

### Predicting Results For Death Cases

In [27]:
death_test_seq  = y_train_death[-1:]
death_preds = []
for i in range(DAYS_TO_PREDICT):
    y2_death = death_test_seq.reshape((len(death_test_seq), time_steps, 1))
    death_pred = death_model.predict(y2_death)
    death_preds.append(death_pred)
    death_new_seq = death_test_seq.flatten()
    death_new_seq = np.append(death_new_seq,[death_pred])
    death_new_seq = death_new_seq[-1:]
    death_test_seq = death_new_seq.reshape((len(death_new_seq), time_steps, 1))
    y2_death = death_test_seq
    
death_preds = (np.array(death_preds).flatten()).reshape(-1,1)

predict_death_index = pd.date_range(start=death_df.index[-1],periods=DAYS_TO_PREDICT + 1,closed='right').strftime('%Y-%m-%d')
predict_death_cases = pd.DataFrame()
predict_death_cases['Dates'] = predict_death_index
predict_death_cases['Death Cases Predictions'] = death_preds
predict_death_cases.set_index('Dates', inplace = True)

In [28]:
fig = go.Figure()
#fig.update_layout(template='plotly_dark')
fig.update_xaxes(tickangle=90, showticklabels = True, type = 'category')
fig.add_trace(go.Scatter(x=predict_death_cases.index, 
                         y=predict_death_cases['Death Cases Predictions'],
                         mode='lines+markers',
                         name='death',
                         line=dict(color='blue', width=2)))

### Predicting Results For Recovered Cases

In [29]:
recovered_test_seq  = y_train_recovered[-1:]
recovered_preds = []
for i in range(DAYS_TO_PREDICT):
    y2_recovered = recovered_test_seq.reshape((len(recovered_test_seq), time_steps, 1))
    recovered_pred = recovered_model.predict(y2_recovered)
    recovered_preds.append(recovered_pred)
    recovered_new_seq = recovered_test_seq.flatten()
    recovered_new_seq = np.append(recovered_new_seq,[recovered_pred])
    recovered_new_seq = recovered_new_seq[-1:]
    recovered_test_seq = recovered_new_seq.reshape((len(recovered_new_seq), time_steps, 1))
    y2_recovered = recovered_test_seq
    
recovered_preds = (np.array(recovered_preds).flatten()).reshape(-1,1)

predict_recovered_index = pd.date_range(start=recovered_df.index[-1],periods=DAYS_TO_PREDICT + 1,closed='right').strftime('%Y-%m-%d')

predict_recovered_cases = pd.DataFrame()
predict_recovered_cases['Dates'] = predict_recovered_index
predict_recovered_cases['Recovered Cases Predictions'] = recovered_preds
predict_recovered_cases.set_index('Dates', inplace = True)

In [31]:
fig = go.Figure()
#fig.update_layout(template='plotly_dark')
fig.update_xaxes(tickangle=90, showticklabels = True, type = 'category')
fig.add_trace(go.Scatter(x=predict_recovered_cases.index, 
                         y=predict_recovered_cases['Recovered Cases Predictions'],
                         mode='lines+markers',
                         name='recovered',
                         line=dict(color='green', width=2)))

# Visualizations

### Historial Cases

In [33]:
fig = go.Figure()
#fig.update_layout(template='plotly_dark')
fig.update_xaxes(tickangle=90, showticklabels = True, type = 'category')
fig.add_trace(go.Scatter(x=confirmed_df.index, 
                         y=confirmed_df[0],
                         mode='lines+markers',
                         name='confirm',
                         line=dict(color='red', width=2)))
fig.add_trace(go.Scatter(x=death_df.index, 
                         y=death_df[0],
                         mode='lines+markers',
                         name='death',
                         line=dict(color='green', width=2)))
fig.add_trace(go.Scatter(x=recovered_df.index, 
                         y=recovered_df[0],
                         mode='lines+markers',
                         name='recovered',
                         line=dict(color='blue', width=2)))

## Predictions for every individual scenario

In [38]:
fig = go.Figure()
#fig.update_layout(template='plotly_dark')
fig.update_xaxes(tickangle=90, showticklabels = True, type = 'category')
fig.add_trace(go.Scatter(x=predicted_confirmed_cases.index, 
                         y=predicted_confirmed_cases['Confirmed Cases Predictions'],
                         mode='lines+markers',
                         name='Confirmed Cases Predictions',
                         line=dict(color='yellow', width=2)))

fig.add_trace(go.Scatter(x=predict_death_cases.index, 
                         y=predict_death_cases['Death Cases Predictions'],
                         mode='lines+markers',
                         name='Death Cases Predictions',
                         line=dict(color='turquoise', width=2)))

fig.add_trace(go.Scatter(x=predict_recovered_cases.index, 
                         y=predict_recovered_cases['Recovered Cases Predictions'],
                         mode='lines+markers',
                         name='Recovered Cases Predictions',
                         line=dict(color='red', width=2)))

## Historical and Predicted Values

In [43]:
fig = go.Figure()
#fig.update_layout(template='plotly_dark')
fig.update_xaxes(tickangle=90, showticklabels = True, type = 'category')
fig.add_trace(go.Scatter(x=confirmed_df.index, 
                         y=confirmed_df[0],
                         mode='lines+markers',
                         name='confirm',
                         line=dict(color='red', width=2)))

fig.add_trace(go.Scatter(x=death_df.index, 
                         y=death_df[0],
                         mode='lines+markers',
                         name='death',
                         line=dict(color='green', width=2)))

fig.add_trace(go.Scatter(x=recovered_df.index, 
                         y=recovered_df[0],
                         mode='lines+markers',
                         name='recovered',
                         line=dict(color='blue', width=2)))

fig.add_trace(go.Scatter(x=predicted_confirmed_cases.index, 
                         y=predicted_confirmed_cases['Confirmed Cases Predictions'],
                         mode='lines+markers',
                         name='Confirmed Cases Predictions',
                         line=dict(color='yellow', width=2)))

fig.add_trace(go.Scatter(x=predict_death_cases.index, 
                         y=predict_death_cases['Death Cases Predictions'],
                         mode='lines+markers',
                         name='Death Cases Predictions',
                         line=dict(color='turquoise', width=2)))

fig.add_trace(go.Scatter(x=predict_recovered_cases.index, 
                         y=predict_recovered_cases['Recovered Cases Predictions'],
                         mode='lines+markers',
                         name='Recovered Cases Predictions',
                         line=dict(color='black', width=2)))
